# Swarm Algorithm for Objects Moving in 2D Domain
The swarm concept in recents years has gain quite popularity in many applications, especially in defence industry. The most attracting side of swarm concept is to be able to fulfill complex duties with relatively cheap ammo.

The code below shows swarming particles (like drones) in 2 Dimensional domain and has been generated with rule based algorithm given in the great paper "Outdoor flocking and formation flight with autonomous aerial robots" by Vicsek et. al. [1] The document referenced in [2] is also a very explanatory and beneficial to understand the basics of swarm concept.

The first thing to do is to import necessary packages to our environment. 

# Refences
[1] "Outdoor flocking and formation flight with autonomous aerial robots", G. Vásárhelyi, Cs. Virágh, G. Somorjai, N. Tarcai, T. Szörényi, T. Nepusz, T. Vicsek

[2] "Dynamic Mission Control for UAV Swarm via Task Stimulus Approach", Haoyang Cheng , John Page, John Olsen

[3] https://github.com/ibrahimkaya754/GeneticAlgorithm

In [7]:
# Import neccessary modules
import time
import math
import matplotlib.pyplot as plt
import pygame
import random
import numpy as np
from pygame.locals import *
from PygameModule import *
from Swarm_Algorithm_RuleBased import *

What is next is, we are deciding the values of some simulation parameters, like the time step size, number of particles swarming, total run time, etc.

In [8]:
# Simulation Parameters
number_of_particles = 51
number_of_axes      = 2
delta_t             = 0.10
t_final             = 100
wght_leader         = np.array((0.0,0.0,0.0,0.0,10.0,0.0))

# Main Parameters
The parameters given below inside the main function has been obtained by Genetic Algorithm (GA). The writers of the paper did not give the values of the parameters, which is quite reasonable to some point, since every application has its own dynamics, i.e., the parameters from one applicastion to the other will probably be very different. For my application, I used GA code I have shared in my github repo [3] and the algorithm has given the parameters seen inside the main function.

There are 2 important modules imported to our environment above. One of them is PygameModule which generates the neccessary things for visualization of the particles moving with swarm condition. The other one is the Swarm_Algoritm_RuleBased, where the rulebased algorithm given in the paper is coded.

What is happenning in the simulation can be summarized as the following:

* There are 51 particles swarming in the domain

* One of them is the leader that every information about the mission is known by

* The other 50 particles does not know anything about the mission but the only thing they know is they should follow the leader without creating any crash inside the population

* The points of the target that is to be followed were given in the list of waypoints, where it can be changed inside the domain randomly.

* The parameters obtained by GA has been generated for a fitness function minimizing the sum of the distances between the group members (minimization of the dispersion), while preventing the individual particles from approaching to eachother greater than a value defined (8 meters given in our case).

* Although the number of total particles (population) has been given as 51, it can be increased (to 100s) or decreased (to around 10) since the algorithm coded can be applied for any number of particles, i.e., the algorithm given here is scalable.

In [9]:
# main function
def main(paramaters=[19.087,77.570,74.741,49.385,50.461,31.121,715.096,
                     87.658,-8.527,9.441,-1.126,7.908,5.326,-3.060]):
    wght_fllwr               = np.array((paramaters[8],paramaters[9],paramaters[10],
                                         paramaters[11],paramaters[12],paramaters[13]),dtype='double')
    distances                = np.zeros((number_of_particles,number_of_particles*number_of_axes))
    distances_abs            = np.zeros((number_of_particles,number_of_particles))
    closest_distances_abs    = np.zeros((number_of_particles,number_of_particles))
    closest_particles_abs    = np.zeros((number_of_particles,number_of_particles),dtype='int32')
    closestneighbours        = np.zeros((number_of_particles),dtype='int16')
    position                 = np.zeros((number_of_particles,number_of_axes)) 
    velocity                 = np.zeros((number_of_particles,number_of_axes)) 
    position_delta           = np.zeros((number_of_particles,number_of_axes)) 
    dist_twp                 = np.zeros((number_of_particles-1,1))

    waypoints   = [[100,50],[300,50],[500,50],[800,75],[120,100],
                   [160,550],[150,550],[100,600],[100,375], [750,375], [165,100], [50,600]]
    waypoint_no = np.random.randint(0,len(waypoints))
    xtrg        = np.array(waypoints[waypoint_no])
    list_min_distance = []
    list_ave_distance = []
        
    screen = pygame.display.set_mode((1200, 700))
    pygame.display.set_caption("Swarm")
    
    background = pygame.Surface(screen.get_size())
    background.fill((255, 255, 255))
    screen.blit(background, (0, 0))
    
    particles = np.zeros((number_of_particles),dtype = particle)
    for ii in range(number_of_particles-2):
        particles[ii]  = particle(screen, background, color =0)
        position[ii,0] = particles[ii].positionx
        position[ii,1] = particles[ii].positiony
        velocity[ii,0] = particles[ii].velx
        velocity[ii,1] = particles[ii].vely
    for ii in range(number_of_particles-2,number_of_particles-1):
        particles[ii]  = particle(screen, background, color =1)
        position[ii,0] = particles[ii].positionx
        position[ii,1] = particles[ii].positiony
        velocity[ii,0] = particles[ii].velx
        velocity[ii,1] = particles[ii].vely
    for ii in range(number_of_particles-1,number_of_particles):
        particles[ii]  = particle(screen, background, color =2)
        particles[ii].positionx = xtrg[0]
        particles[ii].positiony = xtrg[1]
        position[ii,0] = particles[ii].positionx
        position[ii,1] = particles[ii].positiony
        velocity[ii,0] = particles[ii].velx
        velocity[ii,1] = particles[ii].vely
    
    
    dist = distance(population_number=number_of_particles,dimension=number_of_axes,
                    distances=distances,distances_abs=distances_abs,
                    position=position,
                    closest_distances_abs=closest_distances_abs,closest_particles_abs=closest_particles_abs,
                    closestneighbours=closestneighbours)
    
    distances,distances_abs,closest_distances_abs,closest_particles_abs,closestneighbours = dist.find_distances()
    swarm_algo_follower = swarm_algorithm(params=paramaters)
    swarm_algo_leader   = swarm_algorithm(params=paramaters)
    
    allSprites = pygame.sprite.Group(particles[:]) # Grouping the objects to use the uniform method
    clock = pygame.time.Clock()
    time1 = time.process_time()
    keepGoing = True
    iter      = 0
    t         = 0
    counter   = 0

    while keepGoing:
        try:
            xtrg      = np.array(waypoints[waypoint_no]) 
            remainder = iter % 100        
            clock.tick(30)
            for event in pygame.event.get():
                if event.type == QUIT:
                    keepGoing = False
                elif event.type == KEYDOWN and event.key == K_ESCAPE:
                    keepGoing = False
                    
            for ii in range(number_of_particles-2):
                dist_to_ldr = dist.dist_to_wypnt(position[ii],position[number_of_particles-2])
                for jj in range(number_of_axes):
                    swarm_algo_follower.algo(particle=ii,axes=jj,position=position,velocity=velocity,
                                             closest_particles_abs=closest_particles_abs,
                                             xtrg=position[number_of_particles-2],
                                             wght=wght_fllwr,distance_to_target=dist_to_ldr)
                
                    position_delta[ii,jj] = swarm_algo_follower.position_delta
    
                particles[ii].dx = position_delta[ii,0]
                particles[ii].dy = position_delta[ii,1]
                particles[ii].update()
                position[ii,0] = particles[ii].positionx
                position[ii,1] = particles[ii].positiony  
            
            for ii in range(number_of_particles-2,number_of_particles-1):
                dist_to_trg = dist.dist_to_wypnt(position[ii],xtrg)
                for jj in range(number_of_axes): 
                    swarm_algo_leader.algo(particle=ii,axes=jj,position=position,velocity=velocity,
                                           closest_particles_abs=closest_particles_abs,
                                           xtrg=xtrg,wght=wght_leader,distance_to_target=dist_to_trg)
                    
                    position_delta[ii,jj] = swarm_algo_leader.position_delta               
    
                particles[ii].dx = position_delta[ii,0]
                particles[ii].dy = position_delta[ii,1]
                particles[ii].update()
                position[ii,0] = particles[ii].positionx
                position[ii,1] = particles[ii].positiony        
                  
            particles[number_of_particles-1].dx          = 0
            particles[number_of_particles-1].dy          = 0
            particles[number_of_particles-1].update()
            particles[number_of_particles-1].positionx   = xtrg[0]
            particles[number_of_particles-1].positiony   = xtrg[1]                
            particles[number_of_particles-1].rect.center = (particles[number_of_particles-1].positionx, 
                                                            particles[number_of_particles-1].positiony)
            position[number_of_particles-1,0]            = particles[number_of_particles-1].positionx
            position[number_of_particles-1,1]            = particles[number_of_particles-1].positiony
            
            distances,distances_abs,closest_distances_abs, \
            closest_particles_abs,closestneighbours = dist.find_distances()
            dist_twp    = dist.distance_to_waypoint(waypoints[waypoint_no])
            dist_to_trg = dist.dist_to_wypnt(position[number_of_particles-2],xtrg)
            
            iter = iter + 1
            t    = t + delta_t
            counter = counter + 1
            list_min_distance.append(np.min(closest_distances_abs[0:98,1]))
            list_ave_distance.append(np.average(closest_distances_abs[0:98,1:100]))
            
            if dist_to_trg <= 5.0:
                if np.random.rand() >= 0.98:
                    waypoint_no = np.random.randint(0,len(waypoints))
            
            if t >= t_final:
                keepGoing = False
            
            if remainder == 0:
                print('time = ',t,' s ', ' waypoint = ', waypoint_no)
                print('average_min_distance = ', np.average(list_min_distance))
                print('average_group_distance = ', np.average(list_ave_distance))
                print('\n')
                counter = 0
#################################################################################################################################
            allSprites.clear(screen, background)
            allSprites.draw(screen)\

            pygame.display.flip()
        except:
            print('unexpected error --- sorry')
            keepGoing = False
            
    pygame.quit()
    time2 = time.process_time()
    delta_time = time2 - time1
    return position,distances,closestneighbours,delta_time, \
           closest_particles_abs, closest_distances_abs, dist_twp, list_min_distance, list_ave_distance

Below is the cell where the above code is run. 

Although I have run the code for a simulation time of 100 seconds, one can run is as long as he likes.

In [10]:
# run
if __name__ == "__main__":
    position,distances,closestneighbours,delta_time, \
    closest_particles_abs,closest_distances_abs, dist_twp, list_min_distance, list_ave_distance = main()

time =  0.1  s   waypoint =  2
average_min_distance =  10.41177765137998
average_group_distance =  263.7742780461494


time =  10.09999999999998  s   waypoint =  2
average_min_distance =  13.705420535185995
average_group_distance =  218.4602128266907


time =  20.100000000000016  s   waypoint =  2
average_min_distance =  14.454766689596804
average_group_distance =  181.20164095852314


time =  30.100000000000158  s   waypoint =  2
average_min_distance =  13.608209993685385
average_group_distance =  153.12271521687748


time =  40.1000000000003  s   waypoint =  3
average_min_distance =  13.900481359548515
average_group_distance =  140.48185565356616


time =  50.10000000000044  s   waypoint =  3
average_min_distance =  13.898715212168344
average_group_distance =  131.6271530215778


time =  60.100000000000584  s   waypoint =  1
average_min_distance =  12.683553674918901
average_group_distance =  125.34942050841693


time =  70.10000000000029  s   waypoint =  1
average_min_distance =  12